In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch
from operator import itemgetter

In [3]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [4]:
output_parser = StrOutputParser()

In [5]:
prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

In [6]:
# 체인
chain = prompt | llm | output_parser

In [7]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [8]:
science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [9]:
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [10]:
branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행합니다.
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행합니다.
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행합니다.
    general_chain,
)

In [11]:
# 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의합니다.
full_chain = (
    {"topic": chain, "question": itemgetter("question")} | branch | StrOutputParser()
)

In [12]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 두 가지 기초적인 개념인 미분과 적분으로 구성되어 있습니다. 미분은 함수의 변화율을 다루며, 특정 점에서의 기울기를 구하는 방법입니다. 반면, 적분은 함수의 면적을 구하는 방법으로, 주어진 구간에 걸쳐 함수의 값을 더하는 과정을 포함합니다. \n\n미적분은 자연현상, 물리학, 경제학 등 다양한 분야에서 발생하는 문제를 해결하는 데 매우 중요한 도구입니다. 예를 들어, 물체의 속도를 구하거나, 최적화 문제를 해결하는 데 많이 사용됩니다. 미적분의 기본 정리는 미분과 적분이 서로 역과정임을 보여주며, 이는 수학의 여러 분야에서 응용됩니다. \n\n결론적으로, 미적분은 변화와 면적을 다루는 수학의 중요한 영역으로, 많은 실제 문제를 해결하는 데 필수적인 역할을 합니다.'

In [13]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 물체가 지구의 중력에 의해 얼마나 빠르게 떨어지는지를 나타내는 값으로, 일반적으로 지표면에서 약 9.81 m/s²로 알려져 있습니다. 이를 계산하기 위해서는 중력의 법칙을 이용할 수 있습니다. 뉴턴의 만유인력 법칙에 따르면, 두 물체 사이의 중력은 다음과 같이 계산할 수 있습니다:\n\n\\[ F = \\frac{G \\cdot m_1 \\cdot m_2}{r^2} \\]\n\n여기서 \\( F \\)는 두 물체 사이의 중력, \\( G \\)는 만유인력 상수, \\( m_1 \\)과 \\( m_2 \\)는 각각의 물체의 질량, \\( r \\)은 두 물체 사이의 거리입니다. 지구의 중력 가속도를 구하기 위해서는 이 식을 지구와 물체의 질량 및 반지름을 이용해 변형할 수 있습니다. 결국, 지구 표면에서의 중력 가속도는 대략적으로 9.81 m/s²라는 값을 도출할 수 있습니다.'

In [14]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(Retrieval Augmented Generation)는 정보 검색과 생성 모델을 결합한 접근 방식으로, 모델이 외부 데이터베이스나 문서에서 관련 정보를 검색한 후, 이를 바탕으로 자연어 응답을 생성하는 방법입니다. 이를 통해 보다 정확하고 풍부한 답변을 제공할 수 있습니다.'